# 自然语言理解

## 查询数据库

假设有一个程序，让我们输入一个自然语言问题，返回给我们正确的答案:

    a. Which country is Athens in?
    b. Greece.
    
要如何去写这样一个程序呢？首先，让我们假设我们有关于城市和国家的结构化数据：

| City       | Country        | Population |
|------------|----------------|------------|
| athens     | greece         | 1368       |
| bangkok    | thailand       | 1178       |
| barcelona  | spain          | 1280       |
| berlin     | east_germany   | 3481       |
| birmingham | united_kingdom | 1112       |

从这个表格数据中检索答案的最明显的方式是使用 SQL（Structured Query Language，结构化查询语言）编写查询语句：

    SELECT Country FROM city_table WHERE City = 'athens'
    
执行这个查询可以得到结果 greece。那么现在我们的问题是，要怎样才能使用英语得到与我们在查询系统中输入得到的相同效果呢？

第 9 章中描述的**基于特征的文法**形式可以很容易地从英语翻译到 SQL。我们使用 NLTK 自带的文法 sql0.fcfg 来说明如何将句子意思表示与句子分析串联组装。在这个文法中，每个短语的结构规则定义该短语的 SEM 特征值，父节点的 SEM 特征值表示为其子节点 SEM 特征值的组合：

In [1]:
import nltk

nltk.data.show_cfg('grammars/book_grammars/sql0.fcfg')

% start S
S[SEM=(?np + WHERE + ?vp)] -> NP[SEM=?np] VP[SEM=?vp]
VP[SEM=(?v + ?pp)] -> IV[SEM=?v] PP[SEM=?pp]
VP[SEM=(?v + ?ap)] -> IV[SEM=?v] AP[SEM=?ap]
NP[SEM=(?det + ?n)] -> Det[SEM=?det] N[SEM=?n]
PP[SEM=(?p + ?np)] -> P[SEM=?p] NP[SEM=?np]
AP[SEM=?pp] -> A[SEM=?a] PP[SEM=?pp]
NP[SEM='Country="greece"'] -> 'Greece'
NP[SEM='Country="china"'] -> 'China'
Det[SEM='SELECT'] -> 'Which' | 'What'
N[SEM='City FROM city_table'] -> 'cities'
IV[SEM=''] -> 'are'
A[SEM=''] -> 'located'
P[SEM=''] -> 'in'


我们使用该文法来分析语句 What cities are located in China：

In [2]:
from nltk import load_parser

cp = load_parser('grammars/book_grammars/sql0.fcfg')
query = 'What cities are located in China'
trees = list(cp.parse(query.split()))
answer = trees[0].label()['SEM']
answer = [s for s in answer if s]
q = ' '.join(answer)
print(q)

trees[0].draw()

SELECT City FROM city_table WHERE Country="china"


![ch10-tree-1.png](resources/ch10-tree-1.png)

最后，我们在数据库 city.db 上执行查询，检索出结果：

In [3]:
from nltk.sem import chat80

rows = chat80.sql_query('corpora/city_database/city.db', q)
for r in rows:
    print(r[0], end=' ')

canton chungking dairen harbin kowloon mukden peking shanghai sian tientsin 

如果要处理更复杂一点的语句，如 What cities are in China and have populations above 1,000,000，则需要进一步扩展这一文法：

In [4]:
cp = load_parser('grammars/book_grammars/sql1.fcfg')
query = 'What cities are in China and have populations above 1,000,000'
trees = list(cp.parse(query.split()))
answer = trees[0].label()['SEM']
answer = [s for s in answer if s]
q = ' '.join(answer)
print(q)

trees[0].draw()

SELECT City FROM city_table WHERE Country="china" AND Population > 1000


![ch10-tree-2.png](resources/ch10-tree-2.png)

这样的文法很好的满足我们查询数据库的需求，但是它也有很显著的缺点：首先，它“硬生生”地把一些数据库细节加入了其中，我们需要知道有关表和字段的名称，如果数据库中使用的是不同的表名或字段名，SQL 查询就不能执行。其次，我们可以用不同的格式存储我们的数据，如 XML，这种情况下文法需要将我们的语句翻译成 XML 查询语句而不是 SQL，文法的通用性成了一个很大的问题。

这些因素表明我们应该将英语翻译成比 SQL 更加抽象和通用的东西，也就是经典**逻辑**。一旦我们翻译成了逻辑，只要我们想要，就可以再翻译成其他各种特殊用途的语言。

## 自然语言、语义和逻辑

语义中有这样两个基本概念：第一个是在某些情况下，陈述句非真即假；第二个是名词短语和专有名词定义的是世界上的东西。

一旦有了这两种设定，我们就有了一个强大的推理工具，可以判断一个句子集合在某些情况下是否为真。例如：a 中的两个句子可以都为真，而 b 和 c 中的则不能。换句话说，a 中的句子是**一致的**，而 b 和 c 中的是**不一致的**。

    a. Sylvania is to the north of Freedonia.
       Freedonia is a republic.
       
    b. The capital of Freedonia has a population of 9,000.
       No city in Freedonia has a population of 9,000.
       
    c. Sylvania is to the north of Freedonia.
       Freedonia is to the north of Sylvania.
       
从广义上讲，基于逻辑的自然语言语义表示的方法关注那些指导我们判断自然语言的一致性和不一致性的方面。设计一种逻辑语言的句法是为了使这些特征形式更明确，结果是如一致性这样的确定性属性往往可以简化成符号操作，也就是说，一种可以被计算机实施的任务。为了实现这种方法，我们首先要开发一种表示某种可能情况的技术，也就是**模型**。

一个句子集合 W 的**模型**是 W 中所有句子都为真的情况的形式化表示，通常用集合论表示。段落的域 D（我们当前关系的所有实体）是个体的一个集合，而当集合从 D 建立，关系也被确立。让我们看一个具体的例子，域 D 包括 3 个孩子，Stefan、Klaus 和 Evi，分别用 s、k 和 e 表示，记为 D = {s, k, e}。表达式 boy 是包含 Stefan 和 Klaus 的集合，表达式 girl 是包含 Evi 的集合，表达式 is running 是包含 Stefan 和 Evi 的集合。下图是这个模型的图形化描绘：

![model_kids.png](resources/model_kids.png)

在后面的章节中，我们将使用模型来帮助评估英语句子的真假，并用这种方式来说明表示意思的一些方法。然而，在进入更多细节之前，让我们从更广阔的角度来讨论一个问题，一台计算机可以理解句子的意思吗？我们该如何判断它是否能理解？这类似于问“计算机能思考吗？”。阿兰·图灵提出的著名的回答是：通过检查计算机与人类进行理智对话的能力。假设你有一个与人聊天的会话和一个与计算机聊天的会话，但一开始你并不知道哪个是哪个。在与它们两个聊天后，如果你不能识别哪一个事计算机，那么计算机就成功地模仿了人类。如果一台计算机成功地被当作人类通过了这个“图灵测试”，那么就可以说计算机能思考，可以说它具有了智能。所以图灵从侧面回答了这个问题，不是检查计算机的内部状态，而是检查它的行为，作为具有智能的证据。同样的道理，我们认为要说一台计算机懂英语，只需要它的行为表现看上去它懂，也就是以可观察的行为为依据来判断自然语言理解的能力。